In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
accounts = return_account_ids()
api_url = return_api_url()

In [4]:
f"{api_url}/audiences/query"

In [5]:
res = pd.DataFrame()
for acc in accounts:
    print(acc["name"])
    audiences = send_to_innkeepr_api_paginated(
        api_url=f"{api_url}/audiences/query", 
        accountID=acc["id"],
        content={"status":"active"},
        logger=logging
    )
    audiences = pd.json_normalize(audiences)
    if len(audiences) == 0:
        continue
    goals = audiences["config.goal"].unique().tolist()
    goals_resp = call_api_with_accountId(
        endpoint_url=f"{api_url}/goals/query",
        accountID=acc["id"],
        content={"id":goals},
        logger=logging
    )
    goals_resp = pd.json_normalize(goals_resp)
    goals_resp = goals_resp[["id","name","conversionEvents"]]
    audiences = pd.merge(audiences, goals_resp, how="left", left_on="config.goal", right_on="id", suffixes=("","_goal"))
    audiences = audiences[["id","name","config.goal","name_goal","conversionEvents"]]
    audiences["account"] = acc["name"]
    res = pd.concat([res, audiences])
res

In [6]:
temp = res.drop_duplicates(subset=["account","config.goal"]).sort_values(by=["name_goal","account"]).reset_index(drop=True)
temp[["account","name_goal","conversionEvents","config.goal"]]

In [7]:
[{'account': 'Junglueck',
  'conversionActionId': '682b1d8a1de6c855565aeff2',
  'name': 'Adjusted Conversion (purchase) - Innkeepr'
  'goal': '66b5c31047cf247f496ea828'},
 {'account': 'Autobatterienbilliger',
  'conversionActionId': '6876026d768354324df383bd',
  'name': 'Adjusted Conversion (purchase) - Innkeepr', 
  'goal':'6531722cf23edb190c77e696'},
 {'account': 'Nikin',
  'conversionActionId': '6834787b13526dc3d1017e06',
  'name': 'Adjusted Conversion (purchase) - Innkeepr', 
  'goal':'653253af1f8399fb23dd1ac9'},
 {'account': 'Asambeauty',
  'conversionActionId': '682ed8362fc068cde38c3dff',
  'name': 'Adjusted Conversion (purchase) - Innkeepr', 
  'goal':'65ae65e92989649becfd8cf6'},
 {'account': 'Asambeauty',
  'conversionActionId': '687367a7768354324da077a2',
  'name': 'Adjusted Conversion (purchase) - Innkeepr - #2', 
  'goal':'65ae65e92989649becfd8cf6'},
 {'account': 'Ective',
  'conversionActionId': '6876028e768354324df38813',
  'name': 'Adjusted Conversion (purchase) - Innkeepr', 
  'goal':'65326cff1f8399fb23e0a4c7'},
 {'account': 'LILLYDOO',
  'conversionActionId': '68c2d9007bd2ec4485bb98ed',
  'name': 'Synthetic Conversion - Innkeepr', 
  'goal':'65840f17a47b22a123981a35'},
 {'account': 'LILLYDOO',
  'conversionActionId': '68430dd857bdaeb51e1ffc30',
  'name': 'Adjusted Conversion (purchase) - Innkeepr', 
  'goal':'65840f17a47b22a123981a35'},
 {'account': 'JunoAndMe',
  'conversionActionId': '689dda5a97b976aeefb88e6c',
  'name': 'Adjusted Conversion (purchase) - Innkeepr', 
  'goal':'6532600c1f8399fb23dee5e6'},
 {'account': 'Oui',
  'conversionActionId': '68b581502e5f2d9a582baa9a',
  'name': 'Adjusted Conversion (purchase) - Innkeepr, 
  'goal':'67e6c20012398e82e6a5a15e'},
 {'account': 'Kfzteile24',
  'conversionActionId': '68820a2deafeb4bf3b501beb',
  'name': 'Adjusted Conversion (purchase) - Innkeepr, 
  'goal':'681a0bbe879ec174109e6d13'}]